# Gaussian Elimination Method

Solving a system of linear equations using the Gaussian elimination method involves transforming the coefficient matrix of the system into an upper triangular matrix, followed by solving the system using back substitution.

This is one of the most fundamental methods for solving systems of linear equations, especially in cases where the number of unknowns is different from the number of equations.

In [ ]:
from sympy import Matrix, symbols, Eq, latex
from IPython.display import display, Markdown
import sympy as sp

class SymbolicMatrix:
    def __init__(self, matrix, column_labels=None, free_terms=None, display_mode="table"):
        self.matrix = Matrix(matrix)
        self.operations = []
        self.display_mode = display_mode

        if column_labels:
            if len(column_labels) != self.matrix.cols:
                raise ValueError("The number of column labels must be equal to the number of columns in the matrix.")
            self.column_labels = column_labels
        else:
            self.column_labels = [f"x{i+1}" for i in range(self.matrix.cols)]

        if free_terms:
            if len(free_terms) != self.matrix.rows:
                raise ValueError("The number of free terms must be equal to the number of rows in the matrix.")
            self.free_terms = free_terms
        else:
            self.free_terms = [0 for _ in range(self.matrix.rows)]

        display(Markdown("**Initial equations:**"))
        self.display_matrix()

    def set_display_mode(self, mode):
        if mode not in ["table", "equations"]:
            raise ValueError("Display mode must be 'table' or 'equations'.")
        self.display_mode = mode

    def display_matrix(self):
        if self.display_mode == "table":
            self._display_table()
        elif self.display_mode == "equations":
            self._display_equations()

    def _display_table(self):
        matrix_with_free_terms = self.matrix.row_join(Matrix(self.free_terms))
        headers = self.column_labels + ["Free term"]
        display(Markdown(self._matrix_to_markdown(matrix_with_free_terms, headers)))

    def _matrix_to_markdown(self, matrix, headers):
        markdown_str = "| " + " | ".join(headers) + " |\n"
        markdown_str += "| " + " | ".join(["---"] * len(headers)) + " |\n"
        for i in range(matrix.rows):
            row = [str(matrix[i, j]) for j in range(matrix.cols)]
            markdown_str += "| " + " | ".join(row) + " |\n"
        return markdown_str

    def _display_equations(self):
        equations = []
        for i in range(self.matrix.rows):
            terms = []
            for j in range(self.matrix.cols):
                coef = self.matrix[i, j]
                var = symbols(self.column_labels[j])

                # Check if the coefficient is positive, negative, or zero
                if coef > 0:
                    if j == 0:
                        terms.append(f"{latex(coef)} {latex(var)}")
                    else:
                        terms.append(f"+ {latex(coef)} {latex(var)}")
                elif coef < 0:
                    terms.append(f"- {latex(abs(coef))} {latex(var)}")
                elif coef == 0:
                    if j == 0:
                        terms.append(f"0 {latex(var)}")
                    else:
                        terms.append(f"+0 {latex(var)}")

            lhs = " ".join(terms)
            rhs = latex(self.free_terms[i])

            equation = f"{lhs} = {rhs}"
            equations.append(equation)

        display(Markdown("System of equations:"))
        display(Markdown(f"$$\\begin{{aligned}} {self._equations_to_latex(equations)} \\end{{aligned}}$$"))

    def _equations_to_latex(self, equations):
        return " \\\\ ".join(equations)

    def add_row(self, target_row, source_row, coefficient):
        target_idx = self._validate_row_number(target_row)
        source_idx = self._validate_row_number(source_row)

        # Operation on the matrix
        self.matrix.row_op(target_idx, lambda v, j: v + coefficient * self.matrix[source_idx, j])

        # Operation on the free terms
        self.free_terms[target_idx] += coefficient * self.free_terms[source_idx]

        operation_str = f"r{target_row} = r{target_row} + {coefficient}*r{source_row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def multiply_row(self, row, coefficient):
        row_idx = self._validate_row_number(row)

        # Operation on the matrix
        self.matrix.row_op(row_idx, lambda v, _: coefficient * v)

        # Operation on the free terms
        self.free_terms[row_idx] *= coefficient

        operation_str = f"r{row} = {coefficient}*r{row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def swap_rows(self, row1, row2):
        row1_idx = self._validate_row_number(row1)
        row2_idx = self._validate_row_number(row2)

        self.matrix.row_swap(row1_idx, row2_idx)
        self.free_terms[row1_idx], self.free_terms[row2_idx] = self.free_terms[row2_idx], self.free_terms[row1_idx]

        operation_str = f"Swap r{row1} <-> r{row2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def swap_columns(self, col1, col2):
        col1_idx = self._validate_col_number(col1)
        col2_idx = self._validate_col_number(col2)

        self.matrix.col_swap(col1_idx, col2_idx)
        self.column_labels[col1_idx], self.column_labels[col2_idx] = self.column_labels[col2_idx], self.column_labels[col1_idx]

        operation_str = f"Swap col{col1} <-> col{col2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def _validate_row_number(self, row):
        if not isinstance(row, int):
            raise TypeError("Row number must be an integer.")
        if row < 1 or row > self.matrix.rows:
            raise IndexError(f"Row number must be between 1 and {self.matrix.rows}.")
        return row - 1

    def _validate_col_number(self, col):
        if not isinstance(col, int):
            raise TypeError("Column number must be an integer.")
        if col < 1 or col > self.matrix.cols:
            raise IndexError(f"Column number must be between 1 and {self.matrix.cols}.")
        return col - 1

    def print_operations(self):
        display(Markdown("**Performed operations:**"))
        for op in self.operations:
            print(op)

# Example usage:
m = SymbolicMatrix([[1,2,3], [4,5,6], [7,8,12]], column_labels=["x1", "x2", "x3"], free_terms=[10, 11, 18], display_mode="equations")


**Initial equations:**

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 4 x_{1} + 5 x_{2} + 6 x_{3} = 11 \\ 7 x_{1} + 8 x_{2} + 12 x_{3} = 18 \end{aligned}$$

In [ ]:
m.add_row(2, 1, -4)

**Operation:** r2 = r2 + -4*r1

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} - 6 x_{2} - 9 x_{3} = -52 \end{aligned}$$

In [ ]:
m.add_row(3, 1, -7)

**Operation:** r3 = r3 + -7*r1

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 4 x_{1} + 5 x_{2} + 6 x_{3} = 11 \\ 0 x_{1} - 6 x_{2} - 9 x_{3} = -52 \end{aligned}$$

In [ ]:
m.add_row(3, 2, -2)

**Operation:** r3 = r3 + -2*r2

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} +0 x_{2} + 3 x_{3} = 6 \end{aligned}$$

In [ ]:
# We have 3 equations, so we end the operation
# Define symbols
x1, x2, x3 = symbols('x1 x2 x3')
# Define equations
eq1 = Eq(x1 + 2*x2 + 3*x3, 10)
eq2 = Eq(-3*x2 - 6*x3, -29)
eq3 = Eq(3*x3, 6)

In [ ]:
sol_x3 = sp.solve(eq3, x3)[0]
sol_x3

2

In [ ]:
# Substitute solution x3 into equation 2
eq2 = eq2.subs(x3, sol_x3)
# Solve equation 2
sol_x2 = sp.solve(eq2, x2)[0]
sol_x2

17/3

In [ ]:
# Substitute solutions x2 and x3 into equation 1
eq1 = eq1.subs({x2: sol_x2, x3: sol_x3})
# Solve equation 1
sol_x1 = sp.solve(eq1, x1)[0]
sol_x1

-22/3

---

## Exercises for Students

$$
\left\{\begin{array}{c}{x+2 y-2 z=4} \\ {2 x+y+z=0} \\ {3 x+2 y+z=1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y+z-t=2} \\ {2 x+y+z=3} \\ {-x+z-t=0} \\ {3 x+2 y-z+2 t=-1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y-z-t=0} \\ {2 x+3 y-2 z+t=4} \\ {3 x+5 z=0} \\ {-x+y-3 z+2 t=3}\end{array}\right.
$$

In [7]:
## Given System of Equations
import sympy as sp
from sympy import Matrix
x, y, z = sp.symbols('x y z')

In [26]:
equations = [x + 2*y - 2*z - 4 , 2*x + y + z , 3*x + 2*y + z - 1]
equations

[x + 2*y - 2*z - 4, 2*x + y + z, 3*x + 2*y + z - 1]

In [ ]:
#Exercises 1
#Formation of the Augmented Matrix
augmented_matrix = Matrix([[1, 2, -2, 4], [2, 1, 1, 0], [3, 2, 1, 1]])
augmented_matrix

Matrix([
[1, 2, -2, 4],
[2, 1,  1, 0],
[3, 2,  1, 1]])

In [ ]:
augmented_matrix[1, :] = augmented_matrix[1, :] - 2 * augmented_matrix[0, :] # r2 -2*r1
augmented_matrix

Matrix([
[1,  2, -2,  4],
[0, -3,  5, -8],
[3,  2,  1,  1]])

In [ ]:
augmented_matrix[2, :] = augmented_matrix[2, :] - 3 * augmented_matrix[0, :] #r3-3*r1
augmented_matrix

Matrix([
[1,  2, -2,   4],
[0, -3,  5,  -8],
[0, -4,  7, -11]])

In [ ]:
augmented_matrix[2, :] = augmented_matrix[2, :] - (4/3) * augmented_matrix[1, :] #r3-(4/3)*r2
augmented_matrix

Matrix([
[1,  2,                -2,                  4],
[0, -3,                 5,                 -8],
[0,  0, 0.333333333333334, -0.333333333333334]])

In [ ]:
augmented_matrix[2, :] = augmented_matrix[2, :] / augmented_matrix[2, 2] #r3/(1/3)  = 0/(1/3) , 0/(1/3) , (1/3)/(1/3) , -(1/3) / (1/3) = R3 = (0,0,1,-1)
augmented_matrix

Matrix([
[1,  2,  -2,    4],
[0, -3,   5,   -8],
[0,  0, 1.0, -1.0]])

In [ ]:
augmented_matrix[1, :] = augmented_matrix[1, :] - augmented_matrix[1, 2] * augmented_matrix[2, :] #r2-5*r3 = 0-5*0 , -3-5*0 , 5-5*1 , -8-5*-1 = R2 = (0,-3,0,-3)

augmented_matrix

Matrix([
[1,  2,  -2,    4],
[0, -3,   0, -3.0],
[0,  0, 1.0, -1.0]])

In [ ]:
augmented_matrix[1, :] = augmented_matrix[1, :] / augmented_matrix[1, 1] #r2/-3 = 0/-3 , -3/-3 , 0/-3 , -3/-3 = R2 = (0,1,0,1)
augmented_matrix

Matrix([
[1, 2,  -2,    4],
[0, 1,   0,  1.0],
[0, 0, 1.0, -1.0]])

In [ ]:
augmented_matrix[0, :] = augmented_matrix[0, :] - augmented_matrix[0, 2] * augmented_matrix[2, :] #R1-(-2)*R3 = 1-(-2)*0 , 2-(-2)*0 , -2-(-2)*1 , 4-(-2)*-1 = R1 = (1,2,0,2)
augmented_matrix

Matrix([
[1, 2,   0,  2.0],
[0, 1,   0,  1.0],
[0, 0, 1.0, -1.0]])

In [ ]:
augmented_matrix[0, :] = augmented_matrix[0, :] - augmented_matrix[0, 1] * augmented_matrix[1, :] #R1-2*R2 = 1-2*0 , 2-2*1 , 0-2*0 , 2-2*1 = R1 = 1,0,0,0
augmented_matrix

Matrix([
[1, 0,   0,    0],
[0, 1,   0,  1.0],
[0, 0, 1.0, -1.0]])

In [ ]:
solution = augmented_matrix[:, -1] #x=0 , y=1 , z=-1
solution

Matrix([
[   0],
[ 1.0],
[-1.0]])



## Given System of Equations

$$
\begin{cases}
x + 2y - 2z = 4 \quad (1) \\
2x + y + z = 0 \quad (2) \\
3x + 2y + z = 1 \quad (3)
\end{cases}
$$

## 1. Formation of the Augmented Matrix

I converted our system to expanded matrix format

$$
\begin{pmatrix}
1 & 2 & -2 & | & 4 \\
2 & 1 & 1 & | & 0 \\
3 & 2 & 1 & | & 1
\end{pmatrix}
$$

## 2. Forward Elimination

I performed row operations to bring our matrix to upper triangular form.


1. **I updated R2**:
   $$
   R2 = R2 - 2R1 \quad \Rightarrow \quad (2, 1, 1 | 0) - 2(1, 2, -2 | 4) = (0, -3, 5 | -8)
   $$

2. **I updated R3**:
   $$
   R3 = R3 - 3R1 \quad \Rightarrow \quad (3, 2, 1 | 1) - 3(1, 2, -2 | 4) = (0, -4, 7 | -11)
   $$

New Matrix :

$$
\begin{pmatrix}
1 & 2 & -2 & | & 4 \\
0 & -3 & 5 & | & -8 \\
0 & -4 & 7 & | & -11
\end{pmatrix}
$$

**2. Second Pivot (2nd Column)**

1. **I updated R3**:
   $$
   R3 = R3 - \frac{4}{3}R2 \quad \Rightarrow \quad (0, -4, 7 | -11) - \frac{4}{3}(0, -3, 5 | -8) = \left(0, 0, \frac{1}{3} | \frac{1}{3}\right)
   $$

New Matrix:

$$
\begin{pmatrix}
1 & 2 & -2 & | & 4 \\
0 & -3 & 5 & | & -8 \\
0 & 0 & \frac{1}{3} & | & \frac{1}{3}
\end{pmatrix}
$$

## 3. Back Substitution

Now I will perform back substitution with our upper triangular matrix.

**1. Solving the Third Equation**

From the third line I found $z$:
$$
\frac{1}{3}z = \frac{1}{3} \quad \Rightarrow \quad z = 1
$$

**2. Solving the Second Equation**

I placed the value $z$ in the second line:

$$
-3y + 5(1) = -8 \\
-3y + 5 = -8 \\
-3y = -13 \\
y = \frac{13}{3} \approx 4.33
$$

**3. Solving the First Equation**

Finally, I placed the $y$ and $z$ values ​​in the first row:

$$
x + 2\left(\frac{13}{3}\right) - 2(1) = 4 \\
x + \frac{26}{3} - 2 = 4 \\
x + \frac{26}{3} - \frac{6}{3} = 4 \\
x + \frac{20}{3} = 4 \\
x = 4 - \frac{20}{3} = \frac{12}{3} - \frac{20}{3} = -\frac{8}{3} \approx -2.67
$$

## Result

$$
x \approx 0, \quad y = 1, \quad z = -1
$$



In [146]:
## Given System of Equations
import sympy as sp
from sympy import Matrix , solve_linear_system
x, y, z, t = sp.symbols('x y z t')

In [147]:
augmented_matrix = Matrix([[1, 1, 1, -1 , 2], [2, 1, 1, 0, 3], [-1, 0, 1, -1, 0], [3, 2, -1, 2, -1]])
augmented_matrix

Matrix([
[ 1, 1,  1, -1,  2],
[ 2, 1,  1,  0,  3],
[-1, 0,  1, -1,  0],
[ 3, 2, -1,  2, -1]])

In [148]:
augmented_matrix[1, :] = augmented_matrix[1, :] - 2 * augmented_matrix[0, :] # r2 -2*r1
augmented_matrix

Matrix([
[ 1,  1,  1, -1,  2],
[ 0, -1, -1,  2, -1],
[-1,  0,  1, -1,  0],
[ 3,  2, -1,  2, -1]])

In [149]:
augmented_matrix[2, :] = augmented_matrix[2, :] + augmented_matrix[0, :] # r3+r1
augmented_matrix

Matrix([
[1,  1,  1, -1,  2],
[0, -1, -1,  2, -1],
[0,  1,  2, -2,  2],
[3,  2, -1,  2, -1]])

In [150]:
augmented_matrix[3, :] = augmented_matrix[3, :] - 3 * augmented_matrix[0, :] # r4 -3 * r1
augmented_matrix

Matrix([
[1,  1,  1, -1,  2],
[0, -1, -1,  2, -1],
[0,  1,  2, -2,  2],
[0, -1, -4,  5, -7]])

In [151]:
augmented_matrix[2, :] = augmented_matrix[2, :] + augmented_matrix[1, :] # r3+ r2
augmented_matrix

Matrix([
[1,  1,  1, -1,  2],
[0, -1, -1,  2, -1],
[0,  0,  1,  0,  1],
[0, -1, -4,  5, -7]])

In [152]:
augmented_matrix[3, :] = augmented_matrix[3, :] - augmented_matrix[1, :] # r4 -r2
augmented_matrix

Matrix([
[1,  1,  1, -1,  2],
[0, -1, -1,  2, -1],
[0,  0,  1,  0,  1],
[0,  0, -3,  3, -6]])

In [153]:
augmented_matrix[3, :] = augmented_matrix[3, :] +3* augmented_matrix[2, :] # r4 -3*r3
augmented_matrix

Matrix([
[1,  1,  1, -1,  2],
[0, -1, -1,  2, -1],
[0,  0,  1,  0,  1],
[0,  0,  0,  3, -3]])

In [154]:
augmented_matrix[3, :] = augmented_matrix[3, :] / 3 # r4 / 3
augmented_matrix

Matrix([
[1,  1,  1, -1,  2],
[0, -1, -1,  2, -1],
[0,  0,  1,  0,  1],
[0,  0,  0,  1, -1]])

In [155]:
z = 1
t = -1

# -y - z + 2t = -1
# -y - 1 + 2(-1) = -1
# -y - 1 - 2 = -1
# -y - 3 = -1
# -y = 2
# y = -2

In [156]:
z = 1
t = -1
y = -2

x = 2 - (y + z - t)
x = 2 - (-2 + 1 - (-1))
x = 2 - ( -2 + 1 + 1)
x = 2 - 0
x = 2

In [157]:

x = 2
y = -2
z = 1
t = -1


In [159]:
import sympy as sp  #accuracy of the solution
x, y, z, t = sp.symbols('x y z t')

In [160]:
equations = [
    x + y + z - t - 2,
    2*x + y + z - 3,
    -x + z - t,
    3*x + 2*y - z + 2*t + 1
]

In [161]:
solution = sp.solve(equations, (x, y, z, t)) #accuracy of the solution
solution

{t: -1, x: 2, y: -2, z: 1}

## Given System of Equations

The given system is:

$$
\begin{cases}
x + y + z - t = 2 \\
2x + y + z = 3 \\
-x + z - t = 0 \\
3x + 2y - z + 2t = -1
\end{cases}
$$

### Step 1: Form the Augmented Matrix

The system is converted to augmented matrix form:

$$
\begin{pmatrix}
1 & 1 & 1 & -1 & | & 2 \\
2 & 1 & 1 & 0 & | & 3 \\
-1 & 0 & 1 & -1 & | & 0 \\
3 & 2 & -1 & 2 & | & -1
\end{pmatrix}
$$

### Step 2: Gauss Elimination

1. **Updating $ R2 $:**
   $$
   R2 = R2 - 2R1 \quad \Rightarrow \quad (2, 1, 1, 0, | 3) - 2 \times (1, 1, 1, -1, | 2) = (0, -1, -1, 2, | -1)
   $$

   **Updating $ R3 $:**
   $$
   R3 = R3 - R1 \quad \Rightarrow \quad (-1, 0, 1, -1, | 0) + (1, 1, 1, -1, | 2) = (0, 1, 2, -2, | 2)
   $$

   **Updating $ R4 $:**
   $$
   R4 = R4 - 3R1 \quad \Rightarrow \quad (3, 2, -1, 2, | -1) - 3 \times (1, 1, 1, -1, | 2) = (0, -1, -4, 5, | -7)
   $$

   New matrix:

   $$
   \begin{pmatrix}
   1 & 1 & 1 & -1 & | & 2 \\
   0 & -1 & -1 & 2 & | & -1 \\
   0 & 1 & 2 & -2 & | & 2 \\
   0 & -1 & -4 & 5 & | & -7
   \end{pmatrix}
   $$

2. **Updating $ R3 $:**
   $$
   R3 = R3 + R2 \quad \Rightarrow \quad (0, 1, 2, -2, | 2) + (0, -1, -1, 2, | -1) = (0, 0, 1, 0, | 1)
   $$

   **Updating $ R4 $:**
   $$
   R4 = R4 - R2 \quad \Rightarrow \quad (0, -1, -4, 5, | -7) - (0, -1, -1, 2, | -1) = (0, 0, -3, 3, | -6)
   $$

   Updated matrix:

   $$
   \begin{pmatrix}
   1 & 1 & 1 & -1 & | & 2 \\
   0 & -1 & -1 & 2 & | & -1 \\
   0 & 0 & 1 & 0 & | & 1 \\
   0 & 0 & -3 & 3 & | & -3
   \end{pmatrix}
   $$

3. **Updating $ R4 $:**
   $$
   R4 = R4 + 3R3 \quad \Rightarrow \quad (0, 0, -3, 3, | -6) + 3 \times (0, 0, 1, 0, | 1) = (0, 0, 0, 3, | -3)
   $$

   New matrix:

   $$
   \begin{pmatrix}
   1 & 1 & 1 & -1 & | & 2 \\
   0 & -1 & -1 & 2 & | & -1 \\
   0 & 0 & 1 & 0 & | & 1 \\
   0 & 0 & 0 & 1 & | & -1
   \end{pmatrix}
   $$

4. **Scaling $ R4 $:**
   $$
   R4 = R4 / 3 \quad \Rightarrow \quad (0, 0, 0, 1, | -1)
   $$

   Final matrix form:

   $$
   \begin{pmatrix}
   1 & 1 & 1 & -1 & | & 2 \\
   0 & -1 & -1 & 2 & | & -1 \\
   0 & 0 & 1 & 0 & | & 1 \\
   0 & 0 & 0 & 1 & | & -1
   \end{pmatrix}
   $$

### Step 3: Back Substitution

1. From the 4th row: $ t = -1 $
2. From the 3rd row: $ z = 1 $
3. From the 2nd row:
   $$
   -y - z + 2t = -1 \implies -y - 1 + 2(-1) = -1 \implies -y = 2 \implies y = -2
   $$
4. From the 1st row:
   $$
   x + y + z - t = 2 \implies x + (-2) + 1 - (-1) = 2 \implies x = 2
   $$

### Solution

The solution to the system is:

$$
x = 2, \quad y = -2, \quad z = 1, \quad t = -1
$$
